# Protein Design Using a Resfile in PyRosetta

#### Presenter: Jason Klima (klimaj@uw.edu)

In [1]:
from __future__ import print_function

import logging
logging.basicConfig(level=logging.INFO)
import py3Dmol
import pyrosetta
import pyrosetta.toolbox
import pyrosetta.distributed.io

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

AttributeError: 'module' object has no attribute 'DataFrame'

### Initialize PyRosetta with custom command line flags:

In [ ]:
pyrosetta.init("-ignore_unrecognized_res 1 -ex1 -ex2aro -detect_disulf 0")

### For this tutorial, let's use the well-studied native protein crambin from PDB ID 1AB1 (http://www.rcsb.org/structure/1AB1).
### Setup the input pose and scorefunction:

In [ ]:
start_pose = pyrosetta.toolbox.rcsb.pose_from_rcsb("1AB1", ATOM=True, CRYS=False)
pose = start_pose.clone()
scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")

### Make of list of which residues are cysteine:

In [ ]:
cys_res = []
for i, aa in enumerate(start_pose.sequence(), start=1):
    if aa == "C":
        cys_res.append(i)
print(cys_res)

### Inspect `start_pose` using the `py3Dmol` module

In [ ]:
viewer = py3Dmol.view(1200, 800)
viewer.addModels(pyrosetta.distributed.io.to_pdbstring(start_pose), "pdb")
viewer.setStyle({"cartoon": {"color": "spectrum"}, "stick": {"radius": 0.25}})
for i in cys_res:
    viewer.addResLabels({"resi": i})
viewer.zoomTo()
viewer.show()

## *Design strategy:* Design away the cysteine residues (i.e. disulfide bonds) using a resfile, allowing all side-chains to re-pack and all backbone and side-chain torsions to minimize using the `FastDesign` mover.

### Read more about resfile file structure at https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

### To write a resfile, we need to know which chain to mutate.
### We can see that the pose consists of only chain "A" by printing the `pose.pdb_info()` object:

In [ ]:
print(pose.pdb_info())

### More programmatically, we could find which chains are in the `pose` using `pyrosetta.rosetta.core.pose.conf2pdb_chain(pose)` which returns a `pyrosetta.rosetta.std.map_unsigned_long_char` object which is iterable.

In [ ]:
print(pyrosetta.rosetta.core.pose.conf2pdb_chain(pose))

In [ ]:
for k, v in pyrosetta.rosetta.core.pose.conf2pdb_chain(pose).items():
    print(v)

### So we could write a resfile to disc indicating design specifications to mutate only the cysteine residues in chain "A":

In [ ]:
resfile = "./resfile"
with open(resfile, "w") as f:
    f.write("NATAA\n")
    f.write("start\n")
    for i in cys_res:
        f.write("{0} {1} ALLAAxc\n".format(i, pyrosetta.rosetta.core.pose.conf2pdb_chain(pose)[1]))
!cat {resfile}

### Now we can setup the TaskOperations for the `FastDesign` mover. These tell `FastDesign` which residues to design or repack during the packer steps in `FastDesign`.

In [ ]:
# The task factory accepts all the task operations
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

# These are pretty standard
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())

# Include the resfile
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.ReadResfile(resfile))

# Convert the task factory into a PackerTask
packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

### The PackerTask looks as intended! 

### Now we can set up a `MoveMap` or a `MoveMapFactory` to specify which torsions are free to minimize during the minimization steps of the `FastDesign` mover

In [ ]:
# Set up a MoveMapFactory
mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
mmf.all_bb(setting=True)
mmf.all_bondangles(setting=True)
mmf.all_bondlengths(setting=True)
mmf.all_chi(setting=True)
mmf.all_jumps(setting=True)
mmf.set_cartesian(setting=True)

In [ ]:
# Set up a MoveMap
# mm = pyrosetta.rosetta.core.kinematics.MoveMap()
# mm.set_bb(True)
# mm.set_chi(True)
# mm.set_jump(True)

# If needed, you could turn off bb and chi torsions for individual residues like this:

# vector1 of true/false for each residue in the pose
# subset_to_minimize = do_something_set.apply(pose)

# for i in range(1, pose.size() + 1):
#     if (not subset_to_minimize[i]):
#         mm.set_bb(i, False)
#         mm.set_chi(i, False)

### Because some Movers only take as input a `MoveMap`, for backwards-compatibility one could generate a `MoveMap` from a `MoveMapFactory` using `pyrosetta.rosetta.protocols.fold_from_loops.movers.MoveMapFactoryToNamedMoveMapMover()`

### Now let's double-check some more `pose` information to verify that we are ready for `FastDesign`:

In [ ]:
display_pose = pyrosetta.rosetta.protocols.fold_from_loops.movers.DisplayPoseLabelsMover()
display_pose.tasks(tf)
display_pose.movemap_factory(mmf)
display_pose.apply(pose)

### Setting up `FastDesign` prints the default `relaxscript`, showing the `ramp_repack_min` settings with the following assignments:
>ramp_repack_min [scale:fa_rep] [min_tolerance] [coord_cst_weight]

In [ ]:
fast_design = pyrosetta.rosetta.protocols.denovo_design.movers.FastDesign(scorefxn_in=scorefxn, standard_repeats=1)
fast_design.cartesian(True)
fast_design.set_task_factory(tf)
fast_design.set_movemap_factory(mmf)
fast_design.min_type("lbfgs_armijo_nonmonotone") # For non-Cartesian scorefunctions, use "dfpmin_armijo_nonmonotone"
#fast_design.set_movemap(mm) # Could have optionally specified a MoveMap instead of MoveMapFactory
#fast_design.minimize_bond_angles(True) # If not using MoveMapFactory, could specify bond angle minimization here
#fast_design.minimize_bond_lengths(True) # If not using MoveMapFactory, could specify bond length minimization here

### For recommendations on setting `fast_design.min_type()` for the scorefunction being used, see: https://www.rosettacommons.org/docs/latest/rosetta_basics/structural_concepts/minimization-overview#recommendations

### Run FastDesign! Note: this takes ~1min 31s

In [ ]:
%time fast_design.apply(pose)

### Inspect the resulting design!

In [ ]:
viewer = py3Dmol.view(1200, 800)
viewer.addModels(pyrosetta.distributed.io.to_pdbstring(pose), "pdb")
viewer.setStyle({"cartoon": {"color": "spectrum"}, "stick": {"radius": 0.25}})
for i in cys_res:
    viewer.addResLabels({"resi": i})
viewer.zoomTo()
viewer.show()

### By how many Angstroms RMSD did the backbone Cα atoms move?

In [ ]:
pyrosetta.rosetta.core.scoring.CA_rmsd(start_pose, pose)

### What is the delta `total_score` from `start_pose` to `pose`? Why is it large?

In [ ]:
delta_total_score = scorefxn(pose) - scorefxn(start_pose)
print(delta_total_score)

### What is the per-residue energy difference for each mutated position between `start_pose` and `pose`?

In [ ]:
for i in cys_res:
    pose_total_score = pyrosetta.rosetta.protocols.relax.get_per_residue_scores(pose, pyrosetta.rosetta.core.scoring.ScoreType.total_score)[i - 1]
    start_pose_total_score = pyrosetta.rosetta.protocols.relax.get_per_residue_scores(start_pose, pyrosetta.rosetta.core.scoring.ScoreType.total_score)[i - 1]
    print("The delta total_score for residue {} between start_pose and pose is {}".format(i, pose_total_score - start_pose_total_score))